In [56]:
from datetime import datetime

In [57]:
def convert_to_json_serializable(data):
    if isinstance(data, dict):
        # Para diccionarios, aplica la conversión a cada valor
        return {key: convert_to_json_serializable(value) for key, value in data.items()}
    elif isinstance(data, list):
        # Para listas, aplica la conversión a cada elemento
        return [convert_to_json_serializable(element) for element in data]
    elif isinstance(data, datetime):
        # Convierte datetime a string usando isoformat()
        return data.isoformat()
    else:
        # Para cualquier otro tipo de dato, lo devuelve sin cambios
        return data

In [58]:
import pandas as pd
import numpy as np
from faker import Faker
import pyarrow.parquet as pq
import json

faker = Faker('es_ES')

# Provincias de España
provincias = ["Almería", "Cádiz", "Córdoba", "Granada", "Huelva", "Jaén", "Málaga", "Sevilla", "Huesca", "Teruel", "Zaragoza", "Asturias", "Islas Baleares", "Las Palmas", "Santa Cruz de Tenerife", "Cantabria", "Albacete", "Ciudad Real", "Cuenca", "Guadalajara", "Toledo", "Ávila", "Burgos", "León", "Palencia", "Salamanca", "Segovia", "Soria", "Valladolid", "Zamora", "Barcelona", "Girona", "Lleida", "Tarragona", "Badajoz", "Cáceres", "A Coruña", "Lugo", "Ourense", "Pontevedra", "Madrid", "Murcia", "Navarra", "Álava", "Bizkaia", "Gipuzkoa", "La Rioja", "Ceuta", "Melilla", "Alicante", "Castellón", "Valencia"]

# Riesgos, Plazos, Escenarios
riesgos = ["inundación", "riesgo sísmico", "ola de calor"]
plazos = ["corto", "medio", "largo"]
escenarios = ["optimista", "pesimista"]

# Generar datos
data = []

# Función para generar coordenadas de cuadrícula (usado para inundaciones)
def generar_cuadrados(centro, num_cuadrados_por_lado, espaciado, delta):
    puntos_cuadricula = [(i, j) for i in range(num_cuadrados_por_lado) for j in range(num_cuadrados_por_lado)]
    cuadrados = []
    for punto in puntos_cuadricula:
        centro_cuadrado = centro + np.array([punto[0] * espaciado, punto[1] * espaciado]) - np.array([num_cuadrados_por_lado / 2 * espaciado, num_cuadrados_por_lado / 2 * espaciado])
        cuadrado = [
            [centro_cuadrado[0] + delta, centro_cuadrado[1] + delta],  # Superior derecho
            [centro_cuadrado[0] + delta, centro_cuadrado[1] - delta],  # Superior izquierdo
            [centro_cuadrado[0] - delta, centro_cuadrado[1] - delta],  # Inferior izquierdo
            [centro_cuadrado[0] - delta, centro_cuadrado[1] + delta],  # Inferior derecho
        ]
        cuadrados.append(cuadrado)
    return cuadrados

centro = np.array([38.345996, -0.490686])  # Punto central para generación de cuadrados
num_cuadrados_por_lado = 100  # Número de cuadrados por lado
espaciado = 0.001  # Espaciado entre cuadrados
delta = 0.0001  # Tamaño del desplazamiento para los vértices

for provincia in provincias:
    for riesgo in riesgos:
        for plazo in plazos:
            for escenario in escenarios:
                if riesgo == "ola de calor":
                    # Para olas de calor, generamos datos temporales y de temperatura
                    fecha_hora = faker.date_time_between(start_date="-1y", end_date="now")
                    temperatura = np.random.uniform(20, 45)
                    latitud = np.random.uniform(36, 43.5)
                    longitud = np.random.uniform(-9, 3)
                    # Asegúrate de que los datos estén en un formato consistente, como una lista de diccionarios
                    coordenadas = [{"fecha_hora": fecha_hora, "temperatura": temperatura, "latitud": latitud, "longitud": longitud}]

                # Antes de guardar en Parquet, convertir la columna 'Coordenadas' a un string JSON si contiene listas de diccionarios
                data.append([provincia, riesgo, plazo, escenario, coordenadas])

# Crear DataFrame
df = pd.DataFrame(data, columns=["Provincia", "Riesgo", "Plazo", "Escenario", "Coordenadas"])
df['Coordenadas'] = df['Coordenadas'].apply(lambda x: json.dumps(convert_to_json_serializable(x)))
# Guardar como Parquet
df.to_parquet('dataset_riesgos_espana.parquet', partition_cols=['Provincia', 'Riesgo', 'Plazo', 'Escenario'])

print("Dataset generado y guardado como 'dataset_riesgos_espana.parquet'")

Dataset generado y guardado como 'dataset_riesgos_espana.parquet'


In [59]:
data = pd.read_parquet('dataset_riesgos_espana.parquet', engine='pyarrow',
                    )


ArrowNotImplementedError: Unsupported cast from list<element: list<element: double>> to struct using function cast_struct

In [ ]:
data.head()

,Coordenadas,provincia,riesgo,plazo,escenario
